In [5]:
pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [6]:
import logging
import json
from IPython.display import Pretty, HTML
import urllib3

from pyfhirclient.FhirClient import FhirClient

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

# Disable the unverified HTTPS request warnings
urllib3.disable_warnings()

In [7]:
def setup_fhir_client() -> FhirClient:
    fhir_client = FhirClient(
        base_url='https://connectathon.epic.com/Interconnect-Fhir-Basic/api/FHIR/R4/',
        verify_ssl=False,
        extra_headers={
            'Epic-Client-ID': '523f56ca-e7f1-4fe5-961a-13c56f65b9f4'
        })

    fhir_client.auth_type = 'Basic'
    fhir_client.token = 'ZW1wJEZISVJDT046Y3Jvc3NmaGlyMTc='

    return fhir_client

In [8]:
client = setup_fhir_client()

search_results = client.search('Patient', {
    'family': 'Smith',
    'gender': 'male',
    'birthdate': '1965-09-01',
    'address-state': 'NC',
})
Pretty(json.dumps(search_results, indent=4))

ERROR:root:b'""'


HTTPError: 401 Client Error: Unauthorized for url: https://connectathon.epic.com/Interconnect-Fhir-Basic/api/FHIR/R4/Patient?family=Smith&gender=male&birthdate=1965-09-01&address-state=NC

There should be exactly one `Patient` matching the search.

In [ ]:
patient = search_results['entry'][0]
Pretty(json.dumps(patient, indent=4))

In [ ]:
HTML(patient['resource']['text']['div'])

In [ ]:
everything = client.everything('Patient', patient['resource']['id'], count=1000)
Pretty(json.dumps(everything, indent=4))

`MedicationAdministration` should have the medications administered to this Patient.

In [ ]:
medication_administration_results = client.search('MedicationAdministration', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(medication_administration_results, indent=4))

`Observation` should have lab test results, and pregnancy status.

In [ ]:
observation_search_results = client.search('Observation', query_params={
    'patient': patient['resource']['id']
})

Pretty(json.dumps(observation_search_results, indent=4))        

`Encounter` should have hospitalization status.

In [ ]:
encounter_search_results = client.search('Encounter', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(encounter_search_results, indent=4))

In [ ]:
client.close()